<a href="https://colab.research.google.com/github/YoonTae-Hwang/trading_repository/blob/master/Backtrader_DMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Strategy : Dual Moving Average(DMA)

1. DMA Introduction 
 - 주식시장에는 이동평균선(Moving Average)라는 지표가 존재한다. 이 지표는 n일 동안의 주가를 평균낸 값을 이어서 표시하는 지표로서 현재의 주가가 평균으로부터 어느정도 벌어져 있는지를 살펴본다.
 - Moving Average의 등장배경은 금융공학에서 말하는 주식시장의 Brownian Motion과 관련이 있다. 다시 말해, 주가의 방향은 무작위성을 가진다는 것이다. 그럼에도 불구하고, 무작위성을 가지는 움직임의 평균을 통해 어느정도 **방향성**정도는 찾을 수 있을것이라는 생각에 Moving Average가 등장하였다.
 - 주식시장에서 많이 사용되는 이동평균선은 주로 5일, 10일, 20일, 50일, 100일, 200일로 구성되어있다. 이는 주식시장이 평일에만 이루어지기 때문에 일주일의 평균이 아닌 5일의 평균으로 사용한다.
 - 여기서 Dual Moving Average 전략은 앞서 설명한 이동평균선을 2가지를  사용한다. 

2. DMA Method
 - Moving Average에서는 평균을 구할때 다음과 같이 다양한 방법을 사용한다.
    - Simple Moving Average = $\frac{(P_{1}+P_{2}+....+P_{N})}{N}$  
      - 단순이동평균을 구하는 방법이다. 이 때 P는 Closed이다.
    - Weight Moving Average = $\frac{(1 * P_{1} + 2 * P_{2}+....+n * P_{N}}{\frac{n(n+1)}{2}}$
      - 현재에 가까운 가격 데이터가 과거의 데이터보다 더 중요하다는 가정하에 사용한다. 가장 최근의 값인 $P_{n}$에 n만큼의 높은 가중치를 준다. 
    - Exponetial Moving Average = $(P_{n} * EP) + (P_{n-1} * (1-EP))$
      - EP(평활계수 : Exponential Percentage) = $\frac{2}{n+1}$
      - SMA의 단점을 보완하기 위해서 생긴 방법으로 WMA와 유사하게 최근의 가격이 과거의 데이터보다 중요하다는 가정하에 사용한다. 두드러진 특징은 평활 계수가 곱해짐으로써 최근 값에 더 민감해서 단기 거래에 효과적이다.
    - Least Squares Moving Average
    - Cumulative Moving Average
    - Hull Moving Average
    - Wilder Moving Average [링크 달아둘예정]

3. Type of index - typical classification
 1. Leading index : 가격 변동에 선행하는 지표로 미래를 예측하기 위해 사용되며, 이러한 지표들은 보합이나 변동이 거의 없는 상태에서 유용함. 이를 통해서 주식의 가격이 급격히 상승하거나 급격히 하락하는 지점들을 확인할 수 있음.
 2. Lagging index : 가격의 상승 혹은 하락 추세가 지속되는지 약화되는지 확인할 때 사용할 수 있으므로 특정 추세기간에서 유용하게 사용할 수 있다.

4. Type of index - How are the index created?
 1. Oscilators Index : 기술적 지표의 일종으로 과매수와 과매도를 수치적으로 표시한다. 일반적으로 0에서 100까지 값을 갖도록 한다.
 2. Non-Bounded Index: 추세의 강함과 약한정도를 표현해주는 지표로 매수 시점과 매도 시점의 신호를 나타내는대 도움을 준다. 

5. What is Dual Moving averages?
 1. Moving Average는 Lagging index의 일종이다. 즉 이동평균으로 나타나는 신호가 나타나는 순간 이미 실제 추세는 바뀐다. 그렇다고 MA가 전혀 효과가 없는 것이 아니다. 종가를 평활화(Smooth)함으로써 추세의 방향을 더 쉽게 알아 볼 수가 있다.
 2. DMA는 이름그대로 두 가지의 이동평균선을 사용한다. 
    - 단기이동평균선 >= 장기이동평균선 : 강세(Bullish cross over)
    - ![DMA](./strategy_img/DMA.jpg)
    - 단기이동평균선 <= 장기이동평균선 : 약세(Bearish cross over)
    - 단기이동평균선과 장기이동평균선은 적절하게 선택하면 된다.
    - 이 전략은 앞서 설명한 오실레이터(Oscilators index)들과 함께 사용하는 경우에 큰 이점이 있다.

 6. Cautions
    - 과거데이터를 기반으로 하기 때문이 신호가 늦는 경우가 많은데 특히 데이터의 범위가 크면 클수록 지연이 커진다. 또한, 잘못된 신호들이 발생하는 경우가 있으므로 오실레이터 지표들과 함께 사용하는것이 좋다.

 7. Reference paper
  - 

### Before we start

1. Entry :
   - SMA(50) >= SMA(200) : Bullish cross over  
2. Exit : 
   - SMA(50) <= SMA(200) : Bearish cross over
3. Position Sizing :
  - buy / sell Apple 100 shares at time 
4. commission :
  - 0.01

In [62]:
!pip install backtrader
#python library
import datetime
import os.path
import sys
import backtrader as bt
import matplotlib
import matplotlib.pyplot as plt

In [0]:
class SMA(bt.Strategy):

    params = dict(
        pfast=50,  # period for the fast moving average
        pslow=200   # period for the slow moving average
    )
    def __init__(self):
        sma1 = bt.indicators.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.indicators.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
       
        if self.crossover > 0:  # if fast crosses slow to the upside
        
            self.close()
            self.buy(size = 100) # enter long
            print("Buy {} shares".format( self.data.close[0] * 100))
                
 
        elif self.crossover < 0:  # in the market & cross to the downside
           
            self.close()# close long position
            self.sell(size = 100)
            print("Sale {} shares".format(self.data.close[0] * 100))
            

In [71]:
if __name__ == "__main__":
    cerebro = bt.Cerebro()

    cerebro.addstrategy(SMA)

    data = bt.feeds.Quandl(
        dataname = "AAPL",
        fromdate = datetime.datetime(2000,1,1),
        todate = datetime.datetime(2020,4,10),
        buffered = True
    )

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.01)
    cerebro.run()

    portvalue = cerebro.broker.getvalue()
    pnl = portvalue - 100000.0

    print('Final Portfolio Value: ${}'.format(portvalue))
    print('P/L: ${}'.format(pnl))    

    try:
      cerebro.plot(style='candlestick')
    except:
      pass


Buy 152.61030096003 shares
Sale 113.86334875838999 shares
Buy 152.22475914708 shares
Sale 112.64246635071001 shares
Buy 120.35330260974999 shares
Sale 743.58164324612 shares
Buy 973.49307770296 shares
Sale 1554.1190480082 shares
Buy 2411.1797833391 shares
Sale 1630.0707851596003 shares
Buy 1535.6130409865 shares
Sale 6870.4797880009 shares
Buy 6536.6066167484 shares
Sale 10898.630313072 shares
Buy 10415.022226208 shares
Final Portfolio Value: $116176.6302855532
P/L: $16176.630285553198
